# Neural Belief Propagation Auto-Encoder - AE (63,45) Study
This notebook studies the proposed architecture on (63,45) code size. A complexity analysis is conducted.

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-


# sanity check
from platform import python_version
print(f'python: {python_version()}')

import tensorflow as tf
print(f'tensorflow: {tf.version.VERSION}')

physical_devices_available = tf.config.list_physical_devices()
print(f'Available physical devices: {physical_devices_available}')

import os
os.environ['TF_GPU_THREAD_MODE']='gpu_private'

import numpy as np

### Code Size & Reference Code (Optional)
This cell set the code size that will be studied using the argument '(n,k)=(31,16)'.
In this example a reference BCH (31,16) code is used for comparison with the code learned by the AE.
If one does not want to compare the code learned by the AE with such a reference code, the 'use_reference_code' should be set to 'False'.
Different code can be used for reference, although they should have the same size and rate to that of the AE configuration.

In [ ]:
# Study code size
(n,k)=(63,45)

# Get the reference code G and H matrices
use_reference_code = True
if use_reference_code:
    codename = f"BCH_{n}_{k}" # name must match that of a npz file at the "encoders/linearblockencoders_reference/" path
    code_path = os.path.join("./","encoders","linearblockencoders_reference", f"{codename}.npz")

    code_file = np.load(code_path)
    # print(code_file.files)

    G_sys = tf.convert_to_tensor(code_file['G'], dtype=tf.float32)
    H_sys = tf.convert_to_tensor(code_file['H_systematic'], dtype=tf.float32)
    H_nsys = tf.convert_to_tensor(code_file['H_non_systematic'], dtype=tf.float32)

    print("Reference Code Matrices:")
    print("Standard Form Generator Matrix:")
    tf.print(G_sys,summarize=-1)
    print("Standard Form Parity-Check Matrix:")
    tf.print(H_sys,summarize=-1)
    print("Standard Form Parity-Check Matrix:")
    tf.print(H_nsys,summarize=-1)
    
    print("Check that G.H equals to 0:")
    tf.print(tf.matmul(G_sys,tf.transpose(H_sys))%2,summarize=-1)
    tf.print(tf.matmul(G_sys,tf.transpose(H_nsys))%2,summarize=-1)
    
    # Cycle Reduced codes from Nachmani et Al. "Deep Learning for Improved Decoding of Linear Block Codes", as provided by the authors.
    print("CYCLE REDUCED (Nachmani):")
    code_path = os.path.join('./','encoders/linearblockencoders_reference/Nachmani/', f'{codename}_cycle_reduced.npz')
    code_file = np.load(code_path)

    G_cr_sys = tf.convert_to_tensor(code_file['G'], dtype=tf.float32)
    H_cr_sys = tf.convert_to_tensor(code_file['H_systematic'], dtype=tf.float32)
    H_cr_nsys = tf.convert_to_tensor(code_file['H_non_systematic'], dtype=tf.float32)

    print('Reference Cycle Reduced Code Matrices from Nachmani et Al. "Deep Learning for Improved Decoding of Linear Block Codes":')
    print("Standard Form Generator Matrix:")
    tf.print(G_cr_sys,summarize=-1)
    print("Standard Form Parity-Check Matrix:")
    tf.print(H_cr_sys,summarize=-1)
    print("Standard Form Parity-Check Matrix:")
    tf.print(H_cr_nsys,summarize=-1)
    
    print("Check that G.H equals to 0:")
    tf.print(tf.matmul(G_cr_sys,tf.transpose(H_cr_sys))%2,summarize=-1)
    tf.print(tf.matmul(G_cr_sys,tf.transpose(H_cr_nsys))%2,summarize=-1)

### Datasets Generation
The following cell define the training, validation and test datasets generation mechanisms including number of training epochs, steps per epochs, batch size, etc.
While validation and test data are randomly sampled from all possible information words, the training data is sample from the basis, all-zero and all-one vectors.

In [ ]:
from dataset import random_messages_dataset, random_messages_base_all_zero_all_one_dataset
"""
#Dataset logic:
.take() allow to generate a fixed number of batch of the dataset
.cache() allow to use cached data (i.e doesn't execute previous dataset generation instructions)
.shuffle() suffle elements inside batches by picking randomly elmts in a buffer of size buffer_size. similarly to batching procedure, shuffling method of model.fit isn't called when using dataset. it's important to manualy specify a shuffling strategy then.
.prefetch() is used to overlap the processing time of the data producer and data consumer. should be used as last step. 
Here the step before .cache() should be done once and for all at first call of the train_dataset object. The operations after .cache() should be called at each time the dataset is called (ie for each batch in the training).
All the steps before .prefetch() are supposed to be prepared during the training step of the data consumer to be ready when the consumer needs more data.
"""

#TRAIN DATASET
epochs=1000
steps_per_epoch=25
train_batch_size = 64
train_seed = 42
train_dataset = random_messages_base_all_zero_all_one_dataset(
        k, 
        batch=train_batch_size, 
        prefetch=tf.data.AUTOTUNE, 
        seed=train_seed
    ).take(steps_per_epoch * epochs).cache()

#VALIDATION DATASET
validation_seed = 43
val_batch_size = 64
validation_dataset = random_messages_dataset(
        k, 
        batch=val_batch_size, 
        prefetch=tf.data.AUTOTUNE, 
        seed=validation_seed
    ).take(int(8000 / val_batch_size)).cache()

#TEST DATASET
test_batch_size = 100  
test_seed = 44
test_dataset = random_messages_dataset(
        k,
        batch=test_batch_size,  
        prefetch=tf.data.AUTOTUNE,
        seed=test_seed,  
    )


### Noise Settings
The following cell, defines the training and evaluation noise levels ($E_b/N_0$) and compute the corresponding SNR levels based on the code rate.
The training noise level is set to $E_b/N_0 = 4 \mathrm{dB}$ while the model is eevaluated after training between $0$ and $7$ dB.

In [ ]:
from tools import ebno_db_to_snr_db

#Training/eval noise level settings
ebn0_training_dbs = 4
ebn0_eval_dbs = tf.range(0, 7, delta=1, dtype=tf.float32)
noise_power_training_dbs = -ebno_db_to_snr_db(ebn0_training_dbs, k/n)

### Create Study Folders
The following cells creates all the folders necessary to storing the results of the training (BER,BLER,Bit Error Count (BEC) aswell as the corresponding +/-5% confidence intervals, the models parameters and the Tensorboard data visualisation).

Note: Depending on the folder location and model name, the path might exceed the MAX_PATH length on Windows system.
If an utf8 encoding error is observed during the execution of the model, one might need to enable long path in the file system configuration:
- Run Regedit
- navigate to Computer\HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\FileSystem (you can past this into the address bar in Regedit)
- set LongPathsEnabled to 1


In [ ]:
from tools import create_paths_and_summaries

path = path = os.path.join("study-ae-{n}-{k}".format(n=n,k=k))
paths_and_summaries = create_paths_and_summaries(path, 'Eb/N0 (dB)', ebn0_eval_dbs)
summary_ber = paths_and_summaries.summary_ber
summary_bler = paths_and_summaries.summary_bler
summary_bec = paths_and_summaries.summary_bec
summary_blec = paths_and_summaries.summary_blec
summary_bpci_ber = paths_and_summaries.summary_bpci_ber
summary_bpci_bler = paths_and_summaries.summary_bpci_bler
models_path = paths_and_summaries.models_path
tensorboard_path = paths_and_summaries.tensorboard_path

### Define AE Model Configuration
This cell defines the model creation routine that will be called at each model creation of the study protocol. The function should thus expose the various parameters usefull of the study.

In [ ]:
# model definition
from autoencoders import AutoEncoder
from metrics import BitErrorRate, BlockErrorRate, BinomialProportionConfidenceInterval, BitErrorCount, BlockErrorCount

noise_power_training_dbs = -ebno_db_to_snr_db(ebn0_training_dbs, k/n)

def create_model(
    n,                              
    k,                             
    build_dataset,                  
    conf="A",                       
    learning_rate=1e-1,             
    model_index=None,               
    training_noise_power_db=0.0,  
    train_model=True,
    G=None,
    H=None,
    trainable_code=True,
    trainable_decoder=True,
    additional_confs=[],
    name=None,
):
    """
    Model Creation Function

    Args:
        n (int): Code-words size
        k (int): Information block size
        build_dataset (dataset): A dataset with the characteristics of the training/validation dataset must be provided to build the model graph upon creation.
        conf (str, optional) [default="A"]: Type of decoders to be used (see 'decoders/decoder.py' for the different decoder avalaible). Configuration 'A' is the configuration of the decoder as described in the paper. 
        learning_rate (float, optional) [default=1e-1]: Training LR.
        model_index (int, optional) [default=None]: Model index appended to the end of auto-name generation (if name is None).
        training_noise_power_db (float, optional) [default=0.0]: Noise power (in dB) used during training.
        train_model (bool, optional) [default=True]: Wheter to train the model or not.
        G ((k,n) tf.float32 tensor,optional) [default=None]: Generator matrix used for initialisation.
        H ((n-k,n) tf.float32 tensor,optional) [default=None]: Parity-check matrix used for initialisation.
        trainable_code (bool, optional) [default=True]: Whether to train the code of the AE or not (valid if train_model=True).
        trainable_decoder (bool, optional) [default=True]: Whether to train the decoders weights of the AE or not (valid if train_model=True).
        additional_confs (list, optional) [default=[]]: Additional configuration to be tested after the traing of the model defined as a list of parameters that should match that of the create_model() function.
        name (str, optional) [default=None]: Name of the model (that will among other things be used for data storage). If set to None, the model name will be defined following the automatic naming rule defined below.
    """
    
    # Model's name definition
    if name is None:
        conf_string = str(conf) if conf is not None else ""
        lr_string = str(learning_rate) if learning_rate is not None else ""
        index_string = str(model_index) if model_index is not None else ""
        train_model_string = str(train_model) if train_model is not None else ""
        trainable_code_string = (
            str(trainable_code) if trainable_code is not None else ""
        )
        name = f"AE-{n}-{k}-{conf_string}-{index_string}"
    else:
        name = name

    # Auto-Encoder Creation with a fixed number of decoding iteration n_iter=5 
    n_iter = 5
    model = AutoEncoder(
        n,
        k,
        n_iter,
        conf,
        training_noise_power_db=training_noise_power_db,
        G=G,
        H=H,
        trainable_code=trainable_code,
        trainable_decoder=trainable_decoder,
        name=name,
    )

    # List of training and validation metrics
    metric_list = [
            BitErrorRate(name="BER",from_logits=False),
            BlockErrorRate(name="BLER",from_logits=False),
            BitErrorCount(name="BEC",from_logits=False, mode="sum"), 
            BlockErrorCount(name="BLEC",from_logits=False, mode="sum"), 
            BinomialProportionConfidenceInterval(monitor_class=BitErrorRate, monitor_params={"name":"bpci_ber","from_logits":False},fraction=0.95, name="BPCI_BER"),
            BinomialProportionConfidenceInterval(monitor_class=BlockErrorRate, monitor_params={"name":"bpci_bler","from_logits":False},fraction=0.95, name="BPCI_BLER")
        ]
    
    # Model compilation with RMSprop optimizer and BCE loss function.
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=metric_list,
    )

    
    # Build model graph using build dataset (one sample)
    build_datum = list(build_dataset.take(1)) 
    model(build_datum[0][0])

    return model

### Define Training Mechanisms
The train_model function defines the study path where to store results and trained model configuration aswell as the training call-backs to be used during training. Finally it execute the model.fit function and reload the weights of the best model after the training.

In [ ]:
from callbacks.defaults import default_training_callbacks, default_configuration_early_stopping, default_configuration_tensorboard, default_configuration_reduce_lr_on_plateau, default_configuration_model_checkpoint

def train_model(
    model,
    models_path,
    train_dataset,
    validation_dataset,
    tensorboard_path,
    epochs=1000,
    steps_per_epoch=25,
):
    """
    Model Training Function

    Args:
        model (tf model): Model to be trained (must be build and compiled).
        models_path (str): Where to store the model training checkpoints.
        train_dataset (tf dataset): The training dataset to be used to train the model.
        validation_dataset (tf dataset): The validation dataset to be used to monitor the model progress during training.
        tensorboard_path (str): The path for Tensorboard checkpoint (visualistion tool from Tensorflow).
        epochs (int, optional) [default=1000]: Number of training epochs. This correspond to a maximum number as an early stopping call-back is used.
        steps_per_epochs (int, optional) [default=25]: Number of steps per epochs.
    """

    # Define the training call-backs
    ckpt_path = os.path.join(models_path, "checkpoint","checkpoint.tf")
    #configuration_tensorboard = default_configuration_tensorboard(tensorboard_path)
    configuration_earlystopping = default_configuration_early_stopping(
        monitor="loss", patience=200
    )
    configuration_reduce_lr_on_plateau = default_configuration_reduce_lr_on_plateau(
        monitor="val_loss", factor=0.8, patience=50
    )
    print(ckpt_path)
    configuration_model_checkpoint = default_configuration_model_checkpoint(
        filepath=ckpt_path
    )     

    callbacks = default_training_callbacks(
        configuration_earlystopping=configuration_earlystopping,
        #configuration_tensorboard=configuration_tensorboard,
        configuration_reduce_lr_on_plateau=configuration_reduce_lr_on_plateau,
        configuration_model_checkpoint=configuration_model_checkpoint,
    )

    # Start the model's training
    with tf.device("/GPU:0"):
        model.fit(
            x=train_dataset,  
            validation_data=validation_dataset,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            callbacks=callbacks,
            verbose=1,
        )
        
    # Reload best weights at the end of the training
    model.load_weights(ckpt_path)

### Define Evaluation Mechanisms
The evaluate_model function defines the evaluation process of the model.
The ci_condition methods define the evaluation stopping criterion based on confidence intervals.

In [ ]:
from callbacks import BatchTerminationCallback


def ber_ci_condition(_, logs):
    # Return true when the BER confidence interval is smaller than 10% of the estimated BER value
    if 'BPCI_BER' in logs:
        epsilon = 1e-7
        (ci_span, ci_low, ber, ci_high) = logs['BPCI_BER']
        return (ci_span)/(ber+epsilon) < 0.1
    else:
        return False

def bler_ci_condition(_, logs):
    # Return true when the BLER confidence interval is smaller than 10% of the estimated BER value
    if 'BPCI_BLER' in logs:
        epsilon = 1e-7
        (ci_span, ci_low, bler, ci_high) = logs['BPCI_BLER']
        return (ci_span)/(bler+epsilon) < 0.1
    else:
        return False

def evaluate_model(
        summary_ber, 
        summary_bler, 
        summary_bec, 
        summary_blec, 
        summary_bpci_ber, 
        summary_bpci_bler, 
        models_path, 
        model, 
        ebn0_eval_dbs, 
        test_dataset, 
        k, 
        n
    ):
    """
    Model Evaluation Function

    Args:
        summary_ber (Summary): Summary file where to store the BER evaluation metric.
        summary_bler (Summary): Summary file where to store the BLER evaluation metric. 
        summary_bec (Summary): Summary file where to store the (Bit Error Count) BEC evaluation metric. 
        summary_blec (Summary): Summary file where to store the (Block Error Count) BLEC evaluation metric. 
        summary_bpci_ber (Summary): Summary file where to store the (Binomial Proportion Confidence Interval on the BER) BPCI_BER evaluation metric. 
        summary_bpci_bler (Summary): Summary file where to store the (Binomial Proportion Confidence Interval on the BLER) BPCI_BLER evaluation metric. 
        models_path (str): Path where to store the model. 
        model (tf model): Model to be evaluated. 
        ebn0_eval_dbs ([float]): List of Eb/No level to be used for model evaluation. 
        test_dataset (tf dataset): Dataset to be used for evaluation
        k (int): Information block size.
        n (int): Code block size.
    """
    # Initiate the metric list
    bers = []
    blers = []
    becs = []  
    blecs = []
    bpci_bers = [] 
    bpci_blers = []
    
    # mean(Es) assumed to be 1
    noise_power_eval_dbs = -ebno_db_to_snr_db(ebn0_eval_dbs, k / n)
    snr_eval_dbs = ebno_db_to_snr_db(ebn0_eval_dbs, k / n)

    # add SNR values
    summary_ber["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bler["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bec["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_blec["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bpci_ber["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bpci_bler["SNR(dB)"] = snr_eval_dbs.numpy()

    # Evaluate model for each eval Eb/No levels
    for ebn0_eval_db, noise_power_eval_db in zip(ebn0_eval_dbs, noise_power_eval_dbs):
        print(
            f"evaluating {model.name} at Eb/N0 [dB]: {ebn0_eval_db} / N0 [dB]: {noise_power_eval_db}"
        )
        model.channel.noise_power_db=noise_power_eval_db

        termination_callback = BatchTerminationCallback(ber_ci_condition)
        summary = model.evaluate(
            test_dataset,  # validation_dataset,
            steps=25000,
            return_dict=True,
            callbacks=[termination_callback]
        )
        
        (ber_ci_span, ci_min, ber_bpci_metric, ci_max) = summary['BPCI_BER']
        (bler_ci_span, ci_min, bler_bpci_metric, ci_max) = summary['BPCI_BLER']
        
        #print(summary)
        ber = summary["BER"]  
        bler = summary["BLER"] 
        bec = summary["BEC"]  
        blec = summary["BLEC"] 
        bpci_ber = summary["BPCI_BER"]  
        bpci_bler = summary["BPCI_BLER"] 
        bers.append(ber) 
        blers.append(bler)
        becs.append(bec) 
        blecs.append(blec)
        bpci_bers.append(bpci_ber) 
        bpci_blers.append(bpci_bler)
        
        print(f"Eb/N0: {ebn0_eval_db} BER: {ber} 95% CI: {ber_ci_span} - BEC: {bec} - BLER: {ber} 95% CI: {bler_ci_span} - BLEC: {blec}")

    # learned_code
    G, H = model.code_generator(None)

    # Store results and model
    summary_ber[model.name] = bers
    summary_bler[model.name] = blers
    summary_bec[model.name] = becs
    summary_blec[model.name] = blecs
    summary_bpci_ber[model.name] = bpci_bers
    summary_bpci_bler[model.name] = bpci_blers

    model_path = os.path.join(models_path, model.name)
    encoder_path = os.path.join(model_path, model.encoder.name)
    decoder_path = os.path.join(model_path, model.decoder.name)
    code_generator_path = os.path.join(model_path, model.code_generator.name)
    matrices_path = os.path.join(model_path, "matrices")

    print(model_path, encoder_path, decoder_path, code_generator_path)
    os.makedirs(encoder_path, exist_ok=True)
    os.makedirs(decoder_path, exist_ok=True)
    os.makedirs(code_generator_path, exist_ok=True)
    os.makedirs(matrices_path, exist_ok=True)
    print(f" saving model {model.name} in {model_path}")

    model.encoder.save(encoder_path, overwrite=True)
    model.decoder.save(decoder_path, overwrite=True)
    model.code_generator.save(code_generator_path, overwrite=True)
    np.savetxt(os.path.join(matrices_path,"G.csv"), np.array(G), fmt="%i")
    np.savetxt(os.path.join(matrices_path,"H.csv"), np.array(H), fmt="%i")

### Study Protocol
The following cell define the protocol of the study. What are the configuration to be tested and in which order. Then for each configuration the model is created, eventually trained, evaluated and its results stored by successively calling the create_model, train_model and evaluate_model methods. If a model contains additional_confs, then a new model will be tested using the previously learned/used code (e.g. a coding scheme and decoders weights are learned GNBP decoder and then the learned code is evaluated under standard BP decoder).

In [ ]:



from collections import namedtuple
from tools import configurations_product,configurations_list

# List of create_model options
options = ['n','k','build_dataset','conf','learning_rate','model_index','training_noise_power_db','train_model','G','H','trainable_code','trainable_decoder','additional_confs','name']

# Default models configuration (exhaustiv ML, conventional BP and GNBP decoders)
ML_eval_conf = configurations_list(options,[[n,k,train_dataset,'ML',None,0,noise_power_training_dbs,False,None,None,False,False,[],"ML"]])[0]
BP_eval_conf = configurations_list(options,[[n,k,train_dataset,'BP',None,0,noise_power_training_dbs,False,None,None,False,False,[],"BP"]])[0]
GNBP_eval_conf = configurations_list(options,[[n,k,train_dataset,'GNBP',1e-1,0,noise_power_training_dbs,True,None,None,False,True,[],"GNBP"]])[0]

# Number of trial for each training
n_trials= 5

# List of configuration to be tested (following the list of parameters from the create_model method)
config_list = [[n,k,train_dataset,'A',1e-1,i,noise_power_training_dbs,True,None,None,True,True,[BP_eval_conf],f'AE_GNBP_{i}'] for i in range(n_trials)]             
    
if use_reference_code:
    config_list = config_list + [[n,k,train_dataset,'BP',None,0,noise_power_training_dbs,False,G_sys,H_nsys,False,False,[],'BCH_NSYS_BP']]                                                         \
                              + [[n,k,train_dataset,'GNBP',1e-1,i,noise_power_training_dbs,True,G_sys,H_nsys,False,True,[],f'BCH_NSYS_GNBP_{i}'] for i in range(n_trials)]                         \
                              + [[n,k,train_dataset,'BP',None,0,noise_power_training_dbs,False,G_cr_sys,H_cr_nsys,False,False,[],'BCH_NSYS_CR_BP']]                                                \
                              + [[n,k,train_dataset,'GNBP',1e-1,i,noise_power_training_dbs,True,G_cr_sys,H_cr_nsys,False,True,[],f'BCH_NSYS_CR_GNBP_{i}'] for i in range(n_trials)]                \
                              + [[n,k,train_dataset,'BP',None,0,noise_power_training_dbs,False,G_sys,H_sys,False,False,[],'BCH_SYS_BP']]                                                           \
                              + [[n,k,train_dataset,'GNBP',1e-1,i,noise_power_training_dbs,True,G_sys,H_sys,False,True,[],f'BCH_SYS_GNBP_{i}'] for i in range(n_trials)]                 

# Parse all configuration and test them
configurations = configurations_list(options,config_list)
for c in configurations:
    #print(c)
    model = create_model(**c._asdict())
    if c.train_model:
        train_model(model, models_path, train_dataset, validation_dataset, tensorboard_path,epochs,steps_per_epoch)
    (G,H) = model.code_generator(None)
    tf.assert_equal(tf.math.floormod(tf.matmul(G,tf.transpose(H)),2), tf.zeros(shape=(k,(n-k))),message="Generator and PC matrices are not matched as syndrome matrix (G.H^T) is not equal to 0")
    evaluate_model(summary_ber, summary_bler, summary_bec, summary_blec, summary_bpci_ber, summary_bpci_bler, models_path, model, ebn0_eval_dbs, test_dataset, k, n)
    
    # If one configuration contains additional configuration load and test them (using the previously used/learned code)
    if c.additional_confs != []:
        parent_model_name = model.name
        for conf in c.additional_confs:           
            conf = conf._asdict()
            conf['G'] = G
            conf['H'] = H
            conf['name'] = parent_model_name + "_" +conf['name']
            Configuration = namedtuple("Configuration", options)
            conf = Configuration(**conf)
            print("Additionnal Conf:")
            model = create_model(**conf._asdict())
            if conf.train_model:
                train_model(model, models_path, train_dataset, validation_dataset, tensorboard_path, epochs, steps_per_epoch)
            evaluate_model(summary_ber, summary_bler, summary_bec, summary_blec, summary_bpci_ber, summary_bpci_bler, models_path, model, ebn0_eval_dbs, test_dataset, k, n)




Copyright (c) 2022 Orange

Author: Guillaume Larue <guillaume.larue@orange.com>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), 
to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice (including the next paragraph) shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS 
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER 
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE
